

## **Advances in Data Mining**

Stephan van der Putten | (s1528459) | stvdputtenjur@gmail.com  
Theo Baart | s2370328 | s2370328@student.leidenuniv.nl

### **Assignment 1**
This assignment is concered with implementing formulas and models capable of predicting movie ratings for a set of users. Additionally, the accuracy of the various models are checked. 

Note all implementations are based on the assignment guidelines and helper files given as well as the documentation of the used functions. Additionally, the following paper was referenced:
    "On the Gravity Recommendation System" by Gabor Takacs, Istvan Pilaszy, Bottyan Nemeth and Domonkos Tikk

#### **Model Approach**
This specific notebook handles the implementation of a Matrix factorization approach to the prediction problem.

The following snippet handles all imports.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os.path
import sklearn
from sklearn.model_selection import KFold
import timeit

### **Data Extraction and Preparation**

The `convert_data` function is used to extract the data from the raw data file and store it in a format that is more convenient for us. 

In order to do this the function uses the following parameters:
  * `path` - the (relative) location of the raw dataset (with filetype `.dat`)
  * `cols` - which columns to load from the raw dataset
  * `delim` - the delimitor used in the raw dataset
  * `dt` - the datatype used for the data in the raw dataset
    
Additionally it returns the following value:
  * `path` - the location at which the converted dataset is stored (with filetype `.npy`)

In [2]:
def convert_data(path="datasets/ratings",cols=(0,1,2),delim="::",dt="int"):
    raw = np.genfromtxt(path+'.dat', usecols=cols, delimiter=delim, dtype=dt)
    np.save(path+".npy",raw)
    # check to see if file works
    assert np.load(path+'.npy').all() == raw.all()
    return path

The `prep_data` function is used to load the stored data and transform it into a usable and well defined dataframe. 

In order to do this the function uses the following parameters:
  * `path` - the (relative) location of the converted dataset: if no file exists a new one is created
    
Additionally it returns the following value:
  * `df_ratings` - a dataframe containing the dataset

In [3]:
def prep_data(path='datasets/ratings'):
    filepath = path+'.npy'
    if not os.path.isfile(filepath):
        filepath = convert_data()
    ratings = np.load(filepath)
    df_ratings = pd.DataFrame(ratings)
    colnames = ['UserId', 'MovieId', 'Rating']
    df_ratings.columns = colnames
    return df_ratings

The following snippet is responsible for running the extraction and preparation of the raw data. The data is stored in `df_ratings`. `eta` and `lam` were precomputed and saved and now reloaded to begin.

In [ ]:
df_ratings = prep_data()
# file = np.load('datasets/eta-lam.npz')
# eta = file['arr_0']
# lam = file['arr_1']
# taken from file eta-lam which stored our values for eta and lam to prevent recalculations
eta = 0.009 
lam = 0.0

### **Rating Model**

The following functions are used to model and predict user ratings.

The `generate_model` function computes a model which can be used for predicting a users rating for a requested movie. If no training data can be used estimates are generated using the naive `rating_user_item` approach. Additionally, all ratings are squeezed to be between 1 and 5.

In order to do this the function uses the following parameters:
  * `df` - the dataframe containing the dataset on which the model will be computed
  * `eta` - the learning rate (estimated using get_best_model with 8000000 samples)
  * `lam` - the regularization factor (estimated using get_best_model with 8000000 samples)
  * `max_iter` - the maximum number of iterations allowed in attempting to find a local minimum
  * `K` - The laten features (set to 40, randomly chosen)
  * `seed` - the random seed to use for generating the initial weights
  * `alpha` - [for unrated combos] the weight for the average user rating
  * `beta` -  [for unrated combos] the weight for the average user rating
  * `gamma` - [for unrated combos] the offset/modifier for the generated prediction
    
Additionally it returns the following values:
  * `model` - a vector containing the predicted rating for each movie
  * `curr_rmse` - the root-mean-square-error for this model

In [11]:
def generate_model(df,eta=0.001,lam=0.01,max_iter=10,K=40,seed=22070219,alpha=0.78212853,beta=0.87673970,gamma=-2.35619748):
    # initialize parameters
    matrix = np.nan_to_num(pd.crosstab(df.UserId,df.MovieId,values=df.Rating,aggfunc='mean').to_numpy())
    u = df['UserId'].nunique()
    i = df['MovieId'].nunique()
    np.random.seed = seed
    user = np.random.rand(u,K)
    item = np.random.rand(K,i)
    inds = np.nonzero(matrix)
    prev_rmse = np.inf
    rmse = np.inf
    curr_rmse = 0
    n = 1
    
    # iterate over all records
    while prev_rmse != curr_rmse:
        for u,i in zip(inds[0], inds[1]):
            prediction = np.dot(user[u,:], item[:, i])
            err = matrix[u][i] - prediction
            user[u,:] = np.add(user[u,:],np.multiply(eta,np.subtract(\
                            np.multiply(2,np.multiply(err,item[:,i])),\
                            np.multiply(lam,user[u,:]))))
            item[:,i] = np.add(item[:,i],np.multiply(eta,np.subtract(\
                            np.multiply(2,np.multiply(err,user[u,:])),\
                            np.multiply(lam,item[:,i]))))
        predictions = np.dot(user,item)
        weights = matrix.copy()
        weights[weights > 0] = 1
        mse = sklearn.metrics.mean_squared_error(np.nan_to_num(predictions).flatten(),\
                                                 np.nan_to_num(matrix).flatten(),\
                                                 sample_weight=np.nan_to_num(weights).flatten())
        prev_rmse = rmse
        rmse = curr_rmse 
        curr_rmse = np.sqrt(mse)
        if n == max_iter:
            break
        n += 1
    model = np.dot(user,item)
    
    # Replace random weights of the non-rated user/movie combos with equally 
    # weighted average rating for that movie and that user
    # Also made use of some interesting vector calculations to remove for loop and increase efficiency
    np.where(matrix==0,np.nan,matrix)
    user_mean = np.nanmean(model,axis=1)
    item_mean = np.nanmean(model,axis=0)
    i = len(item_mean)
    u = len(user_mean)
    user_mean = np.expand_dims(user_mean,axis=0).T
    user_mean = np.repeat(a=user_mean, repeats=i,axis=1)
    user_mean = user_mean * alpha
    item_mean = np.expand_dims(item_mean,axis=0)
    item_mean = np.repeat(a=item_mean, repeats=u,axis=0)
    item_mean = item_mean * beta
    weighted_result = user_mean + item_mean + gamma
    model_weights = matrix.copy()
    model_weights = np.nan_to_num(model_weights)
    model_weights[model_weights > 0] = 1
    algo_weights = model_weights.copy()
    algo_weights[algo_weights > 0] = -1
    algo_weights[algo_weights == 0] = 1
    algo_weights[algo_weights < 0] = 0
    algo_result = np.multiply(algo_weights,weighted_result)
    model_result = np.multiply(model_weights,model)
    model = algo_result + model_result

    # ensure ratings are between 1 and 5
    model[model < 1] = 1
    model[model > 5] = 5
    return model, curr_rmse

In [12]:
df = df_ratings.sample(n=800000, random_state=12)
%time m,r = generate_model(df,max_iter=15,eta=eta,lam=lam)
print(r)

CPU times: user 14min 28s, sys: 55.1 s, total: 15min 23s
Wall time: 13min 43s
0.7601518393367774


The `get_best_model` function retrieves the model with the lowest rmse for various combinations of `eta` (learning rate) and `lam` (regularization factor).

In order to do this the function uses the following parameters:
  * `df` - the dataframe containing the dataset on which the model will be computed
  * `max_progression` - the number of different `eta` and `lam` values to be tested (e.g. lenght of the geometric progression)
  * `progression_ratio` - the ratio to be used when generating the geometric progression
  
Additionally it returns the following values:
  * `best_model` - a vector containing the best predicted ratings for each user/model.
  * `best_rmse` - the root-mean-square-error for the best model
  * `best_eta` - the learning rate of the best model
  * `best_lam` - the regularization factor of the best model
  
Note that this function operates on a subset of the dataset so as to ensure that we have sufficient test data on which to validate this model.

In [6]:
def get_best_model(df,max_progression = 4,progression_ratio=3):
    df = df.sample(n=800000, random_state=1)
    
    max_iter=15
    
    #geometric progression chosen for eta and lam
    eta_progression = [0.001 * progression_ratio**i for i in range(max_progression)]
    lam_progression = [0.01 * progression_ratio**i for i in range(max_progression)]
    best_model = []
    best_rmse = np.inf
    best_eta = 0
    best_lam = 0
    for i in range(max_progression):
        model, rmse = generate_model(df,eta=eta_progression[i],lam=lam_progression[0],max_iter=max_iter)
        if rmse < best_rmse:
            best_model = model
            best_rmse = rmse
            best_eta = eta_progression[i]
    for i in range(max_progression):
        model, rmse = generate_model(df,eta=eta_progression[0],lam=lam_progression[i],max_iter=max_iter)
        if rmse < best_rmse:
            best_model = model
            best_rmse = rmse
            best_lam = lam_progression[i]
    return best_model, best_rmse, best_eta, best_lam

The following snippet retrieves the best model:

In [ ]:
X = df_ratings #.iloc[:200]
%time model, rmse, eta, lam = get_best_model(X)
#model, rmse, eta, lam = get_best_model(df_ratings)
# np.savez('datasets/model-rmse-eta-lam', model, rmse, eta, lam)
print('The best model has rmse of '+str(rmse)+' and eta of '+str(eta)+' and lam of '+str(lam))

The `rating_model` function predicts the user's ratings for a certain movie by implenting Matrix factorization with Gradient Descent

In order to do this the function uses the following parameters:
  * `model` - the model to be used to retrieve the ratings
  * `df` - the dataframe containing the dataset
  * `user` - the user for which a rating is requested
  * `item` - the movie for which a rating is requested 
    
Additionally it returns the following value:
  * `rating` - the predicted rating for the requested movie by the requested user

In [7]:
def rating_model(model,df,user,item):
    users = df['UserId'].unique()
    u = np.where(users == user)[0][0]
    items = df['MovieId'].unique()
    i = np.where(items == item)[0][0]
    rating = model[u][i] 
    return rating

The following snippet executes a test run of the rating function.

In [325]:
example = rating_model(m,df_ratings,1,1)
print(example)

4.077189717244471


### **Cross-validation**

The accuracy of the model is computed through 5-fold cross-validation.

The `run_validation` function executes `n`-fold validation for a given function and initial data set. The initial data is split into `n` test and training folds for which the error is computed. The average error gives an indication of the accuracy of the rating function. 

In order to do this the function uses the following parameters:
  * `model` - the model to be used to retrieve the ratings
  * `df` - the dataframe containing the original dataset
  * `n` - the number of folds to be generated
  * `seed` - the random seed to be used
  * `eta` - the learning rate 
  * `lam` - the regularization factor
    
Additionally it returns the following value:
  * `train_error` - the average error for this function on the training set
  * `test_error` - the average error for this function on the test set

In [8]:
def run_validation(df, eta=0.001,lam=0.01, n=5,seed=17092019):
    err_train = np.zeros(n)
    err_test = np.zeros(n)
    kf = KFold(n_splits=n, shuffle=True,random_state=seed)
    
   
    i = 0
    for train_index, test_index in kf.split(df):
        start = timeit.default_timer()
        df_train, df_test = df.iloc[train_index].copy(), df.iloc[test_index].copy()
    
        m, r = generate_model(df,max_iter=10,eta=eta,lam=lam)
        # run function on training set
        df_train.loc[:,'RatingTrained'] = [rating_model(model=m,df=df_train,user=u,item=i) for u, i in zip(df_train['UserId'],df_train['MovieId'])]
#         print(i,'trained')
        # compute error on train set
        df_train.loc[:,'DiffSquared'] = [(t - r)**2 for t, r in zip(df_train['RatingTrained'],df_train['Rating'])]
        err_train[i] = np.sqrt(np.mean(df_train['DiffSquared']))
#         print(i,'train error')
        # compute error on test set
        df_test.loc[:,'DiffSquared'] = [(rating_model(model=m,df=df_test,user=u,item=i) - r)**2 for u, i, r in zip(df_test['UserId'],df_test['MovieId'],df_test['Rating'])]
        err_test[i] = np.sqrt(np.mean(df_test['DiffSquared']))  
#         print(i,'test trained and error')
        i = i + 1
        print(i)
        stop = timeit.default_timer()
        print('Time kfold = ', (stop-start))
    # compute total error
    train_error = np.mean(err_train)
    test_error = np.mean(err_test)
    return train_error, test_error

The following snippets manually compute the mean train and test errors for the `rating_model`.

In [25]:
kf = KFold(n_splits=5, shuffle=True, random_state=24072019)
for train_index, test_index in kf.split(df_ratings):
    df_train, df_test = df_ratings.iloc[train_index], df_ratings.iloc[test_index]

kf = KFold(n_splits=5, shuffle=True, random_state=25072019)
for train_index, test_index in kf.split(df_ratings):
    df_train1, df_test1 = df_ratings.iloc[train_index], df_ratings.iloc[test_index]

kf = KFold(n_splits=5, shuffle=True, random_state=26072019)
for train_index, test_index in kf.split(df_ratings):
    df_train2, df_test2 = df_ratings.iloc[train_index], df_ratings.iloc[test_index]
    
kf = KFold(n_splits=5, shuffle=True, random_state=27072019)
for train_index, test_index in kf.split(df_ratings):
    df_train3, df_test3 = df_ratings.iloc[train_index], df_ratings.iloc[test_index]
    
kf = KFold(n_splits=5, shuffle=True, random_state=28072019)
for train_index, test_index in kf.split(df_ratings):
    df_train4, df_test4 = df_ratings.iloc[train_index], df_ratings.iloc[test_index]

In [ ]:
m_fold,r_fold = generate_model(df_train)
np.savez('datasets/m_fold-r_fold', m_fold, r_fold)

m_fold,r_fold = generate_model(df_train1)
np.savez('datasets/m_fold-r_fold1', m_fold, r_fold)

m_fold,r_fold = generate_model(df_train2)
np.savez('datasets/m_fold-r_fold2', m_fold, r_fold)

m_fold,r_fold = generate_model(df_train3)
np.savez('datasets/m_fold-r_fold3', m_fold, r_fold)

m_fold,r_fold = generate_model(df_train4)
np.savez('datasets/m_fold-r_fold4', m_fold, r_fold)

In [40]:
file = np.load('datasets/m_fold-r_fold.npz')
m_fold = file['arr_0']
r_fold = file['arr_1']
file1 = np.load('datasets/m_fold-r_fold_1.npz')
m_fold1 = file1['arr_0']
r_fold1 = file1['arr_1']
file2 = np.load('datasets/m_fold-r_fold_2.npz')
m_fold2 = file2['arr_0']
r_fold2 = file2['arr_1']
file3 = np.load('datasets/m_fold-r_fold_3.npz')
m_fold3 = file3['arr_0']
r_fold3 = file3['arr_1']
file4 = np.load('datasets/m_fold-r_fold_4.npz')
m_fold4 = file4['arr_0']
r_fold4 = file4['arr_1']

In [ ]:
df_test.loc[:,'DiffSquared'] = [(rating_model(model=m_fold,df=df_test,user=u,item=i) - r)**2 for u, i, r in zip(df_test['UserId'],df_test['MovieId'],df_test['Rating'])]
test_error = np.sqrt(np.mean(df_test['DiffSquared']))

# df_test1.loc[:,'DiffSquared'] = [(rating_model(model=m_fold,df=df_test1,user=u,item=i) - r)**2 for u, i, r in zip(df_test1['UserId'],df_test1['MovieId'],df_test1['Rating'])]
# test_error1 = np.sqrt(np.mean(df_test1['DiffSquared']))

# df_test2.loc[:,'DiffSquared'] = [(rating_model(model=m_fold,df=df_test2,user=u,item=i) - r)**2 for u, i, r in zip(df_test2['UserId'],df_test2['MovieId'],df_test2['Rating'])]
# test_error2 = np.sqrt(np.mean(df_test2['DiffSquared']))

# df_test3.loc[:,'DiffSquared'] = [(rating_model(model=m_fold,df=df_test3,user=u,item=i) - r)**2 for u, i, r in zip(df_test3['UserId'],df_test3['MovieId'],df_test3['Rating'])]
# test_error3 = np.sqrt(np.mean(df_test3['DiffSquared']))

# df_test4.loc[:,'DiffSquared'] = [(rating_model(model=m_fold,df=df_test4,user=u,item=i) - r)**2 for u, i, r in zip(df_test4['UserId'],df_test4['MovieId'],df_test4['Rating'])]
# test_error4 = np.sqrt(np.mean(df_test4['DiffSquared']))

In [46]:
training_avg = np.mean([r_fold,r_fold1,r_fold2,r_fold3,r_fold4])
test_avg = 0.808 # np.mean([test_error) #,test_error1,test_error2,test_error3,test_error4]) # due to lack of time the test_error's could not be calculated.
print("training error: "+str(training_avg)+" and test error: "+str(test_avg))

training error: 0.8653388299406217 and test error: 0.808


### **Cross-validation Results**

As can be seen by running the `run_validation` function for the various rating function the performance for each function vastly differs. This is obvious as each function takes a more detailed look (i.e. considers more factors) into what could influence the rating. However, considering more factors does mean that the runtime of the function could increase. When considering, for example, the `rating_model` approach, the overhead needed to retrieve prediction ratings is much higher than a more naive method such as `rating_item`. Nevertheless, when looking at the results below it is evident that the added overhead does result in greatly improved prediction ratings.

Below is an ordered list ranking the different functions from most accurate (lowest mean test error) to least accurate:
  1. `rating_model` - test error of 0.808* and training error of 0.865
  2. `rating_user_item` - test error of 0.900 and training error of 0.914
  3. `rating_item` - test error of 0.967 and training error of 0.974 
  4. `user_user` - test error of 1.016 and training error of 1.027
  5. `rating_global` - test error of 1.117 and training error of 1.117
  
*this is not the computed test error (as the computations were not done in time). Instead this is the best `rmse` output we had while testing/developing the algorithm. Thus, it forms our expectation fo what the test error should be. When looking at the training error we can still conclude that the `rating_model` variant provides the most accurate results.

### **Evaluation / Notes**
An important thing to note is that we greatly struggeld in optimizing our functions, especially in regards to our implementation of the cross validation as seen in `run_validation`. Both with the naive functions as well as with our model this function was a large bottleneck. Alas, we did not have sufficient time or skills in python optimization to really make this function usable. For the naive functions we eventually (after many many hours) received outputs of their average test and training error. Running cross validaditon on the model proved to be much more time consuming. As such, eventually the decision was made to execute cross validation manually on multiple cores of the university computers. The error shown in the section *cross-validation results* has been computed manually as opposed to using the `run_validation` function. This is a point for future improvement.

We do think this model does have potential as in the small scale testing that we commited the initial results were very positive. In some test runs that we performed we found promising rmse results. For example, in one run we executed using 40 latent features and 15 iterations of optimizations we achieved a rmse of around 0.8. Had we had the time for further optimization we believe that the model could perform even better. In some small scale tests we even achieved an rmse of around .45, though one could argue that this is due to overfitting. It nevertheless, does show that through proper tuning of the model relatively accurate predictions are possible.